In [1]:
from keras.layers import Dense
from keras.layers import Conv1D
from keras.layers import Flatten
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import MaxPool1D
from keras.callbacks import EarlyStopping
import numpy as np
import pandas as pd
from keras.layers import Dropout
from keras.layers import SpatialDropout1D
from keras.optimizers import Adam,SGD
from keras.layers import BatchNormalization

Using TensorFlow backend.


In [2]:

visibile = Input(shape=(11,208))
dropout_1 = SpatialDropout1D(0.3)(visibile)

conv_11 = Conv1D(filters=30,kernel_size=3,padding='same',data_format='channels_first',activation='relu')(dropout_1)
#bn_11 = BatchNormalization()(conv_11)
conv_12 = Conv1D(filters=30,kernel_size=3,padding='same',data_format='channels_first',activation='relu')(conv_11)
#bn_12 = BatchNormalization()(conv_12)
spatial_dropout_1 = SpatialDropout1D(0.3)(conv_12)
max_pool_1 = MaxPool1D(pool_size=2,strides=2,data_format='channels_first')(spatial_dropout_1)

conv_21 = Conv1D(filters=30,kernel_size=6,padding='same',data_format='channels_first',activation='relu')(max_pool_1)
#conv_21 = Conv1D(filters=30,kernel_size=6,padding='same',data_format='channels_first',activation='relu')(max_pool_1)
#bn_21 = BatchNormalization()(conv_21)
conv_22 = Conv1D(filters=30,kernel_size=6,padding='same',data_format='channels_first',activation='relu')(conv_21)
#bn_22 = BatchNormalization()(conv_22)
spatial_dropout_2 = SpatialDropout1D(0.3)(conv_22)
max_pool_2 = MaxPool1D(pool_size=2,strides=2,data_format='channels_first')(spatial_dropout_2)

conv_31 = Conv1D(filters=30,kernel_size=9,padding='same',data_format='channels_first',activation='relu')(max_pool_2)
#conv_31 = Conv1D(filters=30,kernel_size=9,padding='same',data_format='channels_first',activation='relu')(max_pool_2)
conv_32 = Conv1D(filters=30,kernel_size=9,padding='same',data_format='channels_first',activation='relu')(conv_31)
spatial_dropout_3 = SpatialDropout1D(0.3)(conv_32)
max_pool_3 = MaxPool1D(pool_size=2,strides=2,data_format='channels_first')(spatial_dropout_3)

conv_41 = Conv1D(filters=30,kernel_size=12,padding='same',data_format='channels_first',activation='relu')(max_pool_3)
#conv_41 = Conv1D(filters=30,kernel_size=12,padding='same',data_format='channels_first',activation='relu')(conv_32)
conv_42 = Conv1D(filters=30,kernel_size=12,padding='same',data_format='channels_first',activation='relu')(conv_41)
spatial_dropout_4 = SpatialDropout1D(0.3)(conv_42)
max_pool_4 = MaxPool1D(pool_size=2,strides=2,data_format='channels_first')(spatial_dropout_4)

#conv_51 = Conv1D(filters=40,kernel_size=3,padding='same',data_format='channels_first',activation='relu')(max_pool_4)
#conv_52 = Conv1D(filters=40,kernel_size=3,padding='same',data_format='channels_first',activation='relu')(conv_51)
#spatial_dropout_2 = SpatialDropout1D(0.5)(conv_22)
#max_pool_5 = MaxPool1D(pool_size=2,strides=2,data_format='channels_first')(conv_52)

flat_1 = Flatten()(max_pool_4)
dropout_2 = Dropout(0.5)(flat_1)

dense_1 = Dense(40,activation='relu')(dropout_2)
#bn_3 = BatchNormalization()(dense_1)
dropout_3 = Dropout(0.5)(dense_1)

dense_2 = Dense(20,activation='relu')(dropout_3)
#bn_4 = BatchNormalization()(dense_2)
dropout_4 = Dropout(0.5)(dense_2)

output = Dense(1,activation='sigmoid')(dropout_4)

model = Model(inputs=visibile, outputs=output)

print(model.summary())


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 11, 208)           0         
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 11, 208)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 30, 208)           1020      
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 30, 208)           2730      
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 30, 208)           0         
_________________________________________________________________
max_pooling1d_1 (MaxP

In [4]:
early_stopping_monitor = EarlyStopping(patience=100)



In [5]:
X_train = np.load('X_train.npy')
y_train = np.load('train_labels.npy')
p = np.random.permutation(len(X_train))
X_train = X_train[p,:,:]
y_train = y_train[p]
print(X_train.shape)
print(y_train.shape)

(500, 11, 208)
(500,)


In [6]:
optimizer = Adam(lr=0.00005)
#optimizer = SGD(lr= 0.00001, momentum=0.9)
model.compile(optimizer=optimizer, loss='binary_crossentropy',metrics=['accuracy'])
model.fit(X_train,y_train,validation_split=0.1, epochs=2000, callbacks=[early_stopping_monitor], batch_size=16)
#model.fit(X_train,y_train,validation_split=0.1, epochs=100)

Instructions for updating:
Use tf.cast instead.
Train on 450 samples, validate on 50 samples
Epoch 1/2000
450/450 [==============================] - 1s 2ms/step - loss: 0.6929 - acc: 0.4933 - val_loss: 0.6925 - val_acc: 0.5200
Epoch 2/2000
450/450 [==============================] - 0s 622us/step - loss: 0.6848 - acc: 0.5133 - val_loss: 0.6926 - val_acc: 0.5000
Epoch 3/2000
450/450 [==============================] - 0s 636us/step - loss: 0.6915 - acc: 0.4778 - val_loss: 0.6923 - val_acc: 0.5800
Epoch 4/2000
450/450 [==============================] - 0s 645us/step - loss: 0.6910 - acc: 0.5022 - val_loss: 0.6914 - val_acc: 0.6400
Epoch 5/2000
450/450 [==============================] - 0s 594us/step - loss: 0.6948 - acc: 0.5000 - val_loss: 0.6912 - val_acc: 0.6200
Epoch 6/2000
450/450 [==============================] - 0s 591us/step - loss: 0.6885 - acc: 0.4867 - val_loss: 0.6895 - val_acc: 0.6800
Epoch 7/2000
450/450 [==============================] - 0s 579us/step - loss: 0.6870 - acc: 0

450/450 [==============================] - 0s 583us/step - loss: 0.6571 - acc: 0.5889 - val_loss: 0.6679 - val_acc: 0.6000
Epoch 59/2000
450/450 [==============================] - 0s 584us/step - loss: 0.6596 - acc: 0.6022 - val_loss: 0.6673 - val_acc: 0.6000
Epoch 60/2000
450/450 [==============================] - 0s 583us/step - loss: 0.6559 - acc: 0.6400 - val_loss: 0.6665 - val_acc: 0.6000
Epoch 61/2000
450/450 [==============================] - 0s 584us/step - loss: 0.6589 - acc: 0.6111 - val_loss: 0.6659 - val_acc: 0.6000
Epoch 62/2000
450/450 [==============================] - 0s 582us/step - loss: 0.6438 - acc: 0.6333 - val_loss: 0.6633 - val_acc: 0.6000
Epoch 63/2000
450/450 [==============================] - 0s 586us/step - loss: 0.6484 - acc: 0.5978 - val_loss: 0.6618 - val_acc: 0.6000
Epoch 64/2000
450/450 [==============================] - 0s 582us/step - loss: 0.6410 - acc: 0.6556 - val_loss: 0.6615 - val_acc: 0.6000
Epoch 65/2000
450/450 [==============================] 

Epoch 118/2000
450/450 [==============================] - 0s 622us/step - loss: 0.6417 - acc: 0.6333 - val_loss: 0.6509 - val_acc: 0.6000
Epoch 119/2000
450/450 [==============================] - 0s 590us/step - loss: 0.6279 - acc: 0.6333 - val_loss: 0.6509 - val_acc: 0.6200
Epoch 120/2000
450/450 [==============================] - 0s 584us/step - loss: 0.6328 - acc: 0.6467 - val_loss: 0.6514 - val_acc: 0.6200
Epoch 121/2000
450/450 [==============================] - 0s 583us/step - loss: 0.6338 - acc: 0.6489 - val_loss: 0.6489 - val_acc: 0.6200
Epoch 122/2000
450/450 [==============================] - 0s 582us/step - loss: 0.6151 - acc: 0.6511 - val_loss: 0.6451 - val_acc: 0.6000
Epoch 123/2000
450/450 [==============================] - 0s 581us/step - loss: 0.6474 - acc: 0.6267 - val_loss: 0.6477 - val_acc: 0.6200
Epoch 124/2000
450/450 [==============================] - 0s 584us/step - loss: 0.6185 - acc: 0.6400 - val_loss: 0.6477 - val_acc: 0.6000
Epoch 125/2000
450/450 [==========

450/450 [==============================] - 0s 582us/step - loss: 0.6079 - acc: 0.6467 - val_loss: 0.6294 - val_acc: 0.6600
Epoch 178/2000
450/450 [==============================] - 0s 582us/step - loss: 0.5996 - acc: 0.6356 - val_loss: 0.6276 - val_acc: 0.6200
Epoch 179/2000
450/450 [==============================] - 0s 586us/step - loss: 0.6244 - acc: 0.6578 - val_loss: 0.6359 - val_acc: 0.6600
Epoch 180/2000
450/450 [==============================] - 0s 585us/step - loss: 0.6227 - acc: 0.6533 - val_loss: 0.6358 - val_acc: 0.6200
Epoch 181/2000
450/450 [==============================] - 0s 583us/step - loss: 0.5908 - acc: 0.6778 - val_loss: 0.6287 - val_acc: 0.6200
Epoch 182/2000
450/450 [==============================] - 0s 582us/step - loss: 0.6092 - acc: 0.6489 - val_loss: 0.6265 - val_acc: 0.6400
Epoch 183/2000
450/450 [==============================] - 0s 582us/step - loss: 0.6063 - acc: 0.6711 - val_loss: 0.6249 - val_acc: 0.6400
Epoch 184/2000
450/450 [=========================

450/450 [==============================] - 0s 582us/step - loss: 0.5565 - acc: 0.7422 - val_loss: 0.5866 - val_acc: 0.7600
Epoch 237/2000
450/450 [==============================] - 0s 585us/step - loss: 0.5844 - acc: 0.7022 - val_loss: 0.5890 - val_acc: 0.7800
Epoch 238/2000
450/450 [==============================] - 0s 585us/step - loss: 0.5787 - acc: 0.7267 - val_loss: 0.5867 - val_acc: 0.8000
Epoch 239/2000
450/450 [==============================] - 0s 586us/step - loss: 0.5545 - acc: 0.7356 - val_loss: 0.5928 - val_acc: 0.7800
Epoch 240/2000
450/450 [==============================] - 0s 583us/step - loss: 0.5530 - acc: 0.7222 - val_loss: 0.5858 - val_acc: 0.7800
Epoch 241/2000
450/450 [==============================] - 0s 586us/step - loss: 0.5823 - acc: 0.7111 - val_loss: 0.5819 - val_acc: 0.7800
Epoch 242/2000
450/450 [==============================] - 0s 582us/step - loss: 0.5594 - acc: 0.7022 - val_loss: 0.5854 - val_acc: 0.8000
Epoch 243/2000
450/450 [=========================

...

450/450 [==============================] - 0s 636us/step - loss: 0.4136 - acc: 0.8178 - val_loss: 0.4131 - val_acc: 0.8800
Epoch 709/2000
450/450 [==============================] - 0s 638us/step - loss: 0.3876 - acc: 0.8400 - val_loss: 0.3984 - val_acc: 0.9000
Epoch 710/2000
450/450 [==============================] - 0s 597us/step - loss: 0.4128 - acc: 0.8444 - val_loss: 0.4004 - val_acc: 0.9000
Epoch 711/2000
450/450 [==============================] - 0s 589us/step - loss: 0.4096 - acc: 0.8156 - val_loss: 0.4063 - val_acc: 0.8800
Epoch 712/2000
450/450 [==============================] - 0s 589us/step - loss: 0.3787 - acc: 0.8467 - val_loss: 0.4049 - val_acc: 0.8800
Epoch 713/2000
450/450 [==============================] - 0s 583us/step - loss: 0.4285 - acc: 0.8289 - val_loss: 0.4309 - val_acc: 0.8400
Epoch 714/2000
450/450 [==============================] - 0s 586us/step - loss: 0.3729 - acc: 0.8667 - val_loss: 0.4275 - val_acc: 0.8400
Epoch 715/2000
450/450 [=========================

450/450 [==============================] - 0s 585us/step - loss: 0.4192 - acc: 0.8444 - val_loss: 0.3810 - val_acc: 0.9200
Epoch 768/2000
450/450 [==============================] - 0s 582us/step - loss: 0.3362 - acc: 0.8600 - val_loss: 0.3818 - val_acc: 0.9000
Epoch 769/2000
450/450 [==============================] - 0s 586us/step - loss: 0.3591 - acc: 0.8600 - val_loss: 0.3785 - val_acc: 0.9000
Epoch 770/2000
450/450 [==============================] - 0s 583us/step - loss: 0.3702 - acc: 0.8578 - val_loss: 0.3743 - val_acc: 0.8800
Epoch 771/2000
450/450 [==============================] - 0s 584us/step - loss: 0.3499 - acc: 0.8556 - val_loss: 0.3818 - val_acc: 0.8800
Epoch 772/2000
450/450 [==============================] - 0s 585us/step - loss: 0.3905 - acc: 0.8444 - val_loss: 0.3861 - val_acc: 0.8600
Epoch 773/2000
450/450 [==============================] - 0s 585us/step - loss: 0.3647 - acc: 0.8444 - val_loss: 0.3804 - val_acc: 0.8800
Epoch 774/2000
450/450 [=========================

450/450 [==============================] - 0s 587us/step - loss: 0.3529 - acc: 0.8533 - val_loss: 0.3737 - val_acc: 0.8800
Epoch 827/2000
450/450 [==============================] - 0s 583us/step - loss: 0.3528 - acc: 0.8733 - val_loss: 0.3626 - val_acc: 0.8800
Epoch 828/2000
450/450 [==============================] - 0s 587us/step - loss: 0.4018 - acc: 0.8400 - val_loss: 0.3592 - val_acc: 0.9000
Epoch 829/2000
450/450 [==============================] - 0s 642us/step - loss: 0.3230 - acc: 0.8533 - val_loss: 0.3712 - val_acc: 0.9000
Epoch 830/2000
450/450 [==============================] - 0s 623us/step - loss: 0.3880 - acc: 0.8289 - val_loss: 0.3728 - val_acc: 0.9000
Epoch 831/2000
450/450 [==============================] - 0s 632us/step - loss: 0.3584 - acc: 0.8267 - val_loss: 0.3638 - val_acc: 0.9000
Epoch 832/2000
450/450 [==============================] - 0s 590us/step - loss: 0.3895 - acc: 0.8378 - val_loss: 0.3688 - val_acc: 0.9000
Epoch 833/2000
450/450 [=========================

450/450 [==============================] - 0s 585us/step - loss: 0.3256 - acc: 0.8533 - val_loss: 0.3537 - val_acc: 0.9200
Epoch 886/2000
450/450 [==============================] - 0s 582us/step - loss: 0.3640 - acc: 0.8489 - val_loss: 0.3554 - val_acc: 0.9200
Epoch 887/2000
450/450 [==============================] - 0s 586us/step - loss: 0.3179 - acc: 0.8644 - val_loss: 0.3464 - val_acc: 0.9200
Epoch 888/2000
450/450 [==============================] - 0s 582us/step - loss: 0.3214 - acc: 0.8622 - val_loss: 0.3449 - val_acc: 0.9000
Epoch 889/2000
450/450 [==============================] - 0s 584us/step - loss: 0.3447 - acc: 0.8622 - val_loss: 0.3434 - val_acc: 0.9200
Epoch 890/2000
450/450 [==============================] - 0s 581us/step - loss: 0.3699 - acc: 0.8356 - val_loss: 0.3522 - val_acc: 0.9000
Epoch 891/2000
450/450 [==============================] - 0s 583us/step - loss: 0.3337 - acc: 0.8956 - val_loss: 0.3551 - val_acc: 0.8800
Epoch 892/2000
450/450 [=========================

450/450 [==============================] - 0s 645us/step - loss: 0.3335 - acc: 0.8556 - val_loss: 0.3517 - val_acc: 0.8800
Epoch 945/2000
450/450 [==============================] - 0s 644us/step - loss: 0.3433 - acc: 0.8644 - val_loss: 0.3431 - val_acc: 0.9200
Epoch 946/2000
450/450 [==============================] - 0s 595us/step - loss: 0.3497 - acc: 0.8444 - val_loss: 0.3508 - val_acc: 0.9200
Epoch 947/2000
450/450 [==============================] - 0s 583us/step - loss: 0.3483 - acc: 0.8622 - val_loss: 0.3525 - val_acc: 0.9200
Epoch 948/2000
450/450 [==============================] - 0s 585us/step - loss: 0.3737 - acc: 0.8556 - val_loss: 0.3756 - val_acc: 0.8400
Epoch 949/2000
450/450 [==============================] - 0s 584us/step - loss: 0.3307 - acc: 0.8711 - val_loss: 0.3686 - val_acc: 0.9000
Epoch 950/2000
450/450 [==============================] - 0s 584us/step - loss: 0.3197 - acc: 0.8778 - val_loss: 0.3612 - val_acc: 0.8800
Epoch 951/2000
450/450 [=========================

In [108]:
import tensorflow as tf
tf.VERSION

'1.13.1'